In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys,os
sys.path.append("..")

import numpy as np
import tensorflow as tf
from example import bert_classifier
from bunch import Bunch
from example import feature_writer, write_to_tfrecords, classifier_processor
from porn_classification import classifier_processor
from data_generator import tokenization
from data_generator import tf_data_utils
from model_io import model_io
import json

tokenizer = tokenization.FullTokenizer(
      vocab_file="/data/xuht/chinese_L-12_H-768_A-12/vocab.txt", 
    do_lower_case=True)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
import jieba
with open("/data/xuht/jd_comment/train.txt", "r") as frobj:
    lines = frobj.read().splitlines()
    topmine_content = []
    all_label = []
    for line in lines:
        content = line.split("__label__")
        if len(content) == 2:
            all_label.append(content[-1])
            label_str = "__label__0"
            content_str = " ".join(list(jieba.cut(content[0])))
            topmine_content.append(" ".join([label_str, content_str])+"\n")
with open("/data/xuht/jd_comment/test.txt", "r") as frobj:
    lines = frobj.read().splitlines()
    for line in lines:
        content = line.split("__label__")
        if len(content) == 2:
            all_label.append(content[-1])
            label_str = "__label__1"
            content_str = " ".join(list(jieba.cut(content[0])))
            topmine_content.append(" ".join([label_str, content_str])+"\n")
            
with open("/data/xuht/jd_comment/topmine.txt", "w") as fwobj:
    for item in topmine_content:
        fwobj.write(item)

In [ ]:
import _pickle as pkl
info = pkl.load(open("/data/xuht/jd_comment/mining_info.pkl", "rb"))
freq_phrases = info["frequent_phrases"]
with open("/data/xuht/jd_comment/phrases.json", "w") as fwobj:
    freq_dict = []
    for item in info["frequent_phrases"]:
        word = item[0]
        freq = item[1]
        tmp = {}
        tmp["word"] = word
        tmp["label"] = "rule"
        freq_dict.append(tmp)
    json.dump(freq_dict, fwobj)

In [ ]:
with open("/notebooks/source/data/topmine/mined_porn_domain_adaptation_v2.txt", "r") as frobj:
    lines = frobj.read().splitlines()
    freq_dict = []
    for line in lines:
        content = line.split("&&&&")
        word = "".join(content[0].split("&"))
        label = "rule"
        tmp = {}
        tmp["word"] = word
        tmp["label"] = "rule"
        freq_dict.append(tmp)
    json.dump(freq_dict, open("/data/xuht/websiteanalyze-data-seqing20180821/rule/phrases.json", "w"))
from data_generator import rule_detector

label_dict = {"label2id":{"正常":0,"rule":1}, "id2label":{0:"正常", 1:"rule"}}
json.dump(label_dict, open("/data/xuht/websiteanalyze-data-seqing20180821/rule/rule_label_dict.json", "w"))

rule_config = {
    "keyword_path":"/data/xuht/websiteanalyze-data-seqing20180821/rule/phrases.json",
    "background_label":"正常",
    "label_dict":"/data/xuht/websiteanalyze-data-seqing20180821/rule/rule_label_dict.json"
}
rule_api = rule_detector.RuleDetector(rule_config)
rule_api.load(tokenizer)

classifier_data_api = classifier_processor.PornClassifierProcessor()
classifier_data_api.get_labels("/data/xuht/websiteanalyze-data-seqing20180821/label_dict.json")

train_examples = classifier_data_api.get_train_examples("/data/xuht/websiteanalyze-data-seqing20180821/seqing_train_20180821")

write_to_tfrecords.convert_classifier_examples_with_rule_to_features(train_examples,
                                                        classifier_data_api.label2id,
                                                        128,
                                                        tokenizer,
                                                        rule_api,
                                                        "/data/xuht/websiteanalyze-data-seqing20180821/rule/train.tfrecords")

test_examples = classifier_data_api.get_train_examples("data/xuht/websiteanalyze-data-seqing20180821/seqing_eval_20180821")
write_to_tfrecords.convert_classifier_examples_with_rule_to_features(test_examples,
                                                        classifier_data_api.label2id,
                                                        128,
                                                        tokenizer,
                                                        rule_api,
                                                        "/data/xuht/websiteanalyze-data-seqing20180821/rule/test.tfrecords")

In [ ]:
from data_generator import rule_detector
label_dict = {"label2id":{"正常":0,"rule":1}, "id2label":{0:"正常", 1:"rule"}}
json.dump(label_dict, open("/data/xuht/jd_comment/rule_label_dict.json", "w"))

In [ ]:
rule_config = {
    "keyword_path":"/data/xuht/jd_comment/phrases.json",
    "background_label":"正常",
    "label_dict":"/data/xuht/jd_comment/rule_label_dict.json"
}
rule_api = rule_detector.RuleDetector(rule_config)
rule_api.load(tokenizer)

In [ ]:
classifier_data_api = classifier_processor.ClassificationProcessor()
classifier_data_api.get_labels("/data/xuht/jd_comment/label_dict.json")

train_examples = classifier_data_api.get_train_examples("/data/xuht/jd_comment/train.txt")

write_to_tfrecords.convert_classifier_examples_with_rule_to_features(train_examples,
                                                        classifier_data_api.label2id,
                                                        128,
                                                        tokenizer,
                                                        rule_api,
                                                        "/data/xuht/jd_comment/rule/train.tfrecords")

test_examples = classifier_data_api.get_train_examples("/data/xuht/jd_comment/test.txt")
write_to_tfrecords.convert_classifier_examples_with_rule_to_features(test_examples,
                                                        classifier_data_api.label2id,
                                                        128,
                                                        tokenizer,
                                                        rule_api,
                                                        "/data/xuht/jd_comment/rule/test.tfrecords")

In [ ]:
from fastText import train_supervised

In [ ]:
with open("/data/xuht/websiteanalyze-data-seqing20180821/seqing_train_20180821", "r") as frobj:
    train_lines = frobj.read().splitlines()
    train_lines = list(set(train_lines))
with open("/data/xuht/websiteanalyze-data-seqing20180821/seqing_test_20180821", "r") as frobj:
    dev_lines = frobj.read().splitlines()
    dev_lines = list(set(dev_lines))

In [ ]:
train_data = []
left_train = []
import jieba, re
for line in train_lines:
    re_pattern = "({}{})".format("__label__", "\d.")
    element_list = re.split(re_pattern, line)
    try:
        text_a = " ".join(list(jieba.cut("".join(element_list[-1].split()))))
        input_labels = int((element_list[1]).split("__label__")[-1])
        if input_labels != 0:
            input_labels = "1"
        else:
            input_labels = "0"
        train_data.append(" ".join([text_a, "__label__{}".format(input_labels)])+"\n")
    except:
        left_train.append(line)
    
eval_data = []
left_eval = []
for line in dev_lines:
    re_pattern = "({}{})".format("__label__", "\d.")
    element_list = re.split(re_pattern, line)
    try:
        text_a = " ".join(list(jieba.cut("".join(element_list[-1].split()))))
        input_labels = int((element_list[1]).split("__label__")[-1])
        if input_labels != 0:
            input_labels = "1"
        else:
            input_labels = "0"
        eval_data.append(" ".join([text_a, "__label__{}".format(input_labels)])+"\n")
    except:
        left_eval.append(line)

In [ ]:
train_data[0:100]

In [ ]:
with open("/data/xuht/websiteanalyze-data-seqing20180821/seqing_train_binary", "w") as fwobj:
    for line in train_data:
        fwobj.write(line)
with open("/data/xuht/websiteanalyze-data-seqing20180821/seqing_eval_binary", "w") as fwobj:
    for line in eval_data:
        fwobj.write(line)

In [ ]:
from fastText import train_supervised

train_data = "/data/xuht/websiteanalyze-data-seqing20180821/seqing_train"
valid_data = "/data/xuht/websiteanalyze-data-seqing20180821/seqing_eval"

# train_supervised uses the same arguments and defaults as the fastText cli
model = train_supervised(
    input=train_data, epoch=25, lr=1.0, wordNgrams=3, verbose=2, minCount=1
)
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))
print_results(*model.test(valid_data))

In [ ]:
eval_data[0:10]

In [ ]:

1876/3176, 1952/2753

In [ ]:
671/679,1292/1561,697/965,1561/3990,965/7260,3990/11752

In [ ]:
1935/3176

In [7]:
s= [0.1,0.2]
mean(s)

NameError: name 'mean' is not defined

In [3]:
with open("/notebooks/source/data/topmine/mined_porn_domain_adaptation_v2.txt", "r") as frobj:
    lines = frobj.read().splitlines()
    freq_dict = []
    for line in lines:
        content = line.split("&&&&")
        word = "".join(content[0].split("&"))
        label = "rule"
        tmp = {}
        tmp["word"] = word
        tmp["label"] = "rule"
        freq_dict.append(tmp)
    json.dump(freq_dict, open("/data/xuht/websiteanalyze-data-seqing20180821/rule/phrases.json", "w"))
from data_generator import rule_detector

label_dict = {"label2id":{"正常":0,"rule":1}, "id2label":{0:"正常", 1:"rule"}}
json.dump(label_dict, open("/data/xuht/websiteanalyze-data-seqing20180821/rule/rule_label_dict.json", "w"))

rule_config = {
    "keyword_path":"/data/xuht/websiteanalyze-data-seqing20180821/rule/phrases.json",
    "background_label":"正常",
    "label_dict":"/data/xuht/websiteanalyze-data-seqing20180821/rule/rule_label_dict.json"
}
rule_api = rule_detector.RuleDetector(rule_config)
rule_api.load(tokenizer)

{'label': 'rule', 'word': '爆菊'}
=succeeded in loading keywords== 7256


In [4]:
for item in freq_dict:
    if item["word"] == "爆操":
        print(1)

In [11]:
freq_dict

[{'label': 'rule', 'word': '床电影'},
 {'label': 'rule', 'word': '19禁情色电影'},
 {'label': 'rule', 'word': '阴唇外翻'},
 {'label': 'rule', 'word': '抚弄阴蒂'},
 {'label': 'rule', 'word': '股阴精'},
 {'label': 'rule', 'word': '奶子两'},
 {'label': 'rule', 'word': '热舞视频'},
 {'label': 'rule', 'word': '根粗大肉棒'},
 {'label': 'rule', 'word': '找小姐包夜电话'},
 {'label': 'rule', 'word': '屄湿'},
 {'label': 'rule', 'word': '龟头雨点'},
 {'label': 'rule', 'word': '聊污私'},
 {'label': 'rule', 'word': '肉棒插进'},
 {'label': 'rule', 'word': '鸡巴真'},
 {'label': 'rule', 'word': '播丨笣'},
 {'label': 'rule', 'word': '一下阴道'},
 {'label': 'rule', 'word': '你妈逼操你妈逼操你妈逼操'},
 {'label': 'rule', 'word': '国语对白'},
 {'label': 'rule', 'word': '尿穴里'},
 {'label': 'rule', 'word': '柔软胴体'},
 {'label': 'rule', 'word': '好看美眉激情'},
 {'label': 'rule', 'word': '下体插入'},
 {'label': 'rule', 'word': '人体性交'},
 {'label': 'rule', 'word': '抽送2'},
 {'label': 'rule', 'word': '美白皂身体乳晕嫩红素快速阴唇乳头'},
 {'label': 'rule', 'word': '小姐找全套包夜上门服务小姐电'},
 {'label': 'rule', 'word': '杨静小穴里'}

In [8]:
with open("/notebooks/source/data/topmine/mined_porn_domain_adaptation_v2.txt", "r") as frobj:
    lines = frobj.read().splitlines()
    freq_dict = []
    for line in lines:
        content = line.split("&&&&")
        word = "".join(content[0].split("&"))
        freq_dict.append(word)
        
with open("/notebooks/source/data/topmine/odps_green_kfc_v2.txt", "w") as fwobj:
    for word in freq_dict:
        fwobj.write(word+"\n")

In [9]:
s= [1,2,3]
d = json.dumps(s)

In [10]:
d

'[1, 2, 3]'

In [13]:
with open("/data/xuht/websiteanalyze-data-seqing20180821/label_dict.json", "r") as frobj:
    data = json.load(frobj)

In [14]:
data

{'id2label': {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4'},
 'label2id': {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}}

In [37]:
1402171/24/4*2

29211.895833333332

4.583333333333333